In [ ]:
""" MODELO PRE ENTRENADO VIT base
Dataset: Patch Camelyon
Fuente del modelo: HuggingFace
Modelo: google/vit-base-patch16-224-in21k
"""

' MODELO PRE ENTRENADO VIT base\nDataset: Patch Camelyon copmleto\nFuente del modelo: HuggingFace\nModelo: google/vit-base-patch16-224-in21k\n'

In [ ]:
# Conectar con Google Drive y HuggingFace
from google.colab import drive
_ = drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"

from google.colab import userdata
userdata.get('WANDB_API_KEY')

Mounted at /content/drive


'564b0488f378c4295871f444da8efc813a0e5e49'

In [ ]:
# Instalar las librerías necesarias
!pip install tf-keras
!pip install --upgrade transformers
!pip install --upgrade evaluate
!pip install -q tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import io
import os

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_pil_image

import tensorflow as tf
import tensorflow_datasets as tfds

from transformers import AutoModelForImageClassification
from transformers import AutoImageProcessor
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

from PIL import Image

In [ ]:
# Clase para convertir el dataset de tensorflow en un dataset de torch
class TfToTorchvisionDataset(Dataset):
    def __init__(self, tf_dataset):
        self.data = list(tf_dataset)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_item = self.data[idx]
        image = torch.from_numpy(data_item['image'].numpy())
        image = image.permute(2,0,1)
        label = torch.tensor(data_item['label'].numpy())
        return image, label

In [ ]:
# Cargar en memoria los datasets de training y val:
ruta_pcam_tr = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_train"
ds_tr = tf.data.Dataset.load(ruta_pcam_tr)

torch_dataset_tr = TfToTorchvisionDataset(ds_tr)
ds_tr=None

ruta_pcam_val = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_val"
ds_val = tf.data.Dataset.load(ruta_pcam_val)
torch_dataset_val = TfToTorchvisionDataset(ds_val)
ds_val=None

In [ ]:
# Instanciar el modelo pre-entrenado
model_name = "google/vit-base-patch16-224-in21k"
# Definir el número de clases
num_labels = 2


In [ ]:
# Instanciar el procesador de imágenes
image_processor = AutoImageProcessor.from_pretrained(model_name,do_rescale=True)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
train_dataset = torch_dataset_tr
val_dataset = torch_dataset_val

In [ ]:
# Función de ensamble y preparción de lotes para el Trainer
def collate_fn(batch):
    images = [to_pil_image(item[0]) for item in batch]
    labels = [item[1] for item in batch]
    inputs = image_processor(images=images, return_tensors="pt")
    labels = torch.tensor(labels,dtype=torch.long)
    return {"pixel_values": inputs["pixel_values"], "labels": labels}

In [ ]:
# Instanciar el modelo pre-entrenado para clasificación de imágenes
model = AutoModelForImageClassification.from_pretrained(
               model_name,num_labels=num_labels, ignore_mismatched_sizes=True)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Definir los argumentos de entrenamiento en la clase training_args
training_args = TrainingArguments(
    output_dir=base_folder+"/Modelos entrenados/ViT base v1",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=100,
    remove_unused_columns=False,
    report_to="none",
)

In [ ]:
#Definir la métrica (exactitud)
metric = evaluate.load("accuracy")
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    references = p.label_ids
    metrics_result = metric.compute(predictions=predictions, references=references)
    print(f"Metric computation result: {metrics_result}")
    return {"eval_accuracy": metrics_result["accuracy"]}

In [ ]:
 # Instanciar el Trainer:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)


In [ ]:
print(f"Número de ejemplos de entrenamiento: {len(train_dataset)}")
print(f"Número de ejemplos de prueba: {len(val_dataset)}")


Número de ejemplos de entrenamiento: 262144
Número de ejemplos de prueba: 32768


In [ ]:
# Ejecutar el fine-tuning
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.057700,0.337405,0.902344


Metric computation result: {'accuracy': 0.90234375}


TrainOutput(global_step=8192, training_loss=0.10364334759105986, metrics={'train_runtime': 4501.4729, 'train_samples_per_second': 58.235, 'train_steps_per_second': 1.82, 'total_flos': 2.0314060125527605e+19, 'train_loss': 0.10364334759105986, 'epoch': 1.0})

In [ ]:
# EVALUACION DEL MODELO

# Cargar el Dataset de test:
ruta_pcam_test = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_test"
ds_test = tf.data.Dataset.load(ruta_pcam_test)
torch_dataset_test = TfToTorchvisionDataset(ds_test)
ds_test = None

print(f"Número de ejemplos de test: {len(torch_dataset_test)}")

# Evaluar el modelo en el conjunto de test
eval_results = trainer.evaluate(eval_dataset=torch_dataset_test)

print("Resultados de la evaluación en el conjunto de test:")
eval_results


Metric computation result: {'accuracy': 0.886474609375}


In [ ]:
# Guardar el modelo y los pesos
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
processor.save_pretrained(base_folder+"/Modelos entrenados/google-vit-base v1")
model.save_pretrained(base_folder+"/Modelos entrenados/google-vit-base v1")

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
# Definir nuevos argumentos de entrenamiento para una época adicional
additional_training_args = TrainingArguments(
    output_dir=base_folder+"/Modelos entrenados/ViT base v1_epoca2",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=10,
    remove_unused_columns=False,
    report_to="none",
)

# Instanciar el nuevo Trainer con los pesos actuales y los nuevos argumentos
trainer_additional_epoch = Trainer(
    model=model,
    args=additional_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

trainer_additional_epoch.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.045000,0.420452,0.894073


Metric computation result: {'accuracy': 0.894073486328125}


TrainOutput(global_step=8192, training_loss=0.060873842153228, metrics={'train_runtime': 4514.8313, 'train_samples_per_second': 58.063, 'train_steps_per_second': 1.814, 'total_flos': 2.0314060125527605e+19, 'train_loss': 0.060873842153228, 'epoch': 1.0})

In [ ]:
ruta_pcam_test = base_folder+"/Datasets/pcamelyon_tf/TFcamelyon_test"
ds_test = tf.data.Dataset.load(ruta_pcam_test)
torch_dataset_test = TfToTorchvisionDataset(ds_test)
ds_test = None

print(f"Número de ejemplos de test: {len(torch_dataset_test)}")

# Evaluar el modelo en el conjunto de test
eval_results = trainer.evaluate(eval_dataset=torch_dataset_test)

print("Resultados de la evaluación en el conjunto de test:")
eval_results


Número de ejemplos de test: 32768


Metric computation result: {'accuracy': 0.883575439453125}
Resultados de la evaluación en el conjunto de test:


{'eval_accuracy': 0.883575439453125,
 'eval_loss': 0.4455232620239258,
 'eval_runtime': 224.7071,
 'eval_samples_per_second': 145.825,
 'eval_steps_per_second': 4.557,
 'epoch': 1.0}

In [ ]:
# Guardar el modelo y los pesos
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
processor.save_pretrained(base_folder+"/Modelos entrenados/google-vit-base v1")
model.save_pretrained(base_folder+"/Modelos entrenados/google-vit-base v1")

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
